In [ ]:
import torch
from torch import nn
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import re

class RNN_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(input_size=1080, hidden_size=240, num_layers=1, nonlinearity='relu', bias=True)
        self.output = nn.Linear(in_features=240, out_features=24)

    def forward(self, x):
        y, hidden = self.rnn(x)
        x = self.output(y)
        return x

class DiseasePredictor:
    def __init__(self, model_path):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")
        
        self.model = RNN_model()
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        self.model = self.model.to(self.device)  # Move model to device
        self.model.eval()
        self.all_symptoms = [
            'fever', 'cough', 'fatigue', 'difficulty breathing', 'headache',
            'body ache', 'sore throat', 'runny nose', 'nausea', 'vomiting',
            'diarrhea', 'chest pain', 'abdominal pain', 'joint pain', 'rash',
            'chills', 'sweating', 'loss of appetite', 'weight loss', 'dizziness',
            'muscle weakness', 'confusion', 'blurred vision', 'numbness', 'tingling',
            'swelling', 'constipation', 'anxiety', 'depression', 'insomnia',
            'memory problems', 'tremors', 'seizures', 'shortness of breath', 'wheezing',
            'itching', 'dry mouth', 'excessive thirst', 'frequent urination', 'hair loss',
            'bruising', 'bleeding', 'pale skin', 'jaundice', 'muscle cramps'
        ]
        
        self.mlb = MultiLabelBinarizer()
        self.mlb.fit([self.all_symptoms])
        
        self.disease_mapping = {
            0: "Common Cold",
            1: "Influenza",
            2: "Pneumonia",
            3: "Bronchitis",
            4: "Asthma",
            5: "COVID-19",
            6: "Sinusitis",
            7: "Gastroenteritis",
            8: "Food Poisoning",
            9: "Migraine",
            10: "Hypertension",
            11: "Diabetes",
            12: "Arthritis",
            13: "Allergies",
            14: "Urinary Tract Infection",
            15: "Thyroid Disorder",
            16: "Anemia",
            17: "Depression",
            18: "Anxiety",
            19: "Insomnia",
            20: "Eczema",
            21: "Psoriasis",
            22: "Celiac Disease",
            23: "Irritable Bowel Syndrome"
        }
    
    def extract_symptoms(self, text):
        text = text.lower()
        found_symptoms = []
        
        text = re.sub(r'[^\w\s]', ' ', text)
        words = text.split()
        
        for symptom in self.all_symptoms:
            if symptom in text:
                found_symptoms.append(symptom)
            
        return found_symptoms
    
    def preprocess_symptoms(self, symptoms_text):
        extracted_symptoms = self.extract_symptoms(symptoms_text)
        
        if not extracted_symptoms:
            raise ValueError("No recognized symptoms found in the input text.")
        
        binary_symptoms = self.mlb.transform([extracted_symptoms])
        
        padded_symptoms = np.pad(
            binary_symptoms, 
            ((0, 0), (0, 1080 - binary_symptoms.shape[1])),
            'constant'
        )
        
        symptoms_tensor = torch.FloatTensor(padded_symptoms).unsqueeze(1)
        symptoms_tensor = symptoms_tensor.to(self.device)  # Move tensor to same device as model
        
        return symptoms_tensor
    
    def predict_disease(self, symptoms_text):
        try:
            if not symptoms_text or not isinstance(symptoms_text, str):
                raise ValueError("Please provide a valid symptoms description.")

            symptoms_tensor = self.preprocess_symptoms(symptoms_text)

            # Get prediction
            with torch.no_grad():
                output = self.model(symptoms_tensor)
                output = output.squeeze(0).squeeze(0)  
                probabilities = torch.softmax(output, dim=0)  
                predicted_idx = torch.argmax(probabilities).item()

            predicted_disease = self.disease_mapping.get(predicted_idx, "Unknown Disease")

            return {
                "disease": predicted_disease,
                "recognized_symptoms": self.extract_symptoms(symptoms_text)
            }

        except Exception as e:
            return f"Error in prediction: {str(e)}"
    
def main():
    print("Initializing Disease Prediction System...")
    predictor = DiseasePredictor('pretrained_symtom_to_disease_model.pth')
    
    print("\nDisease Prediction System Ready!")
    print("--------------------------------")
    print("Tips for describing symptoms:")
    print("- Be specific about what you're experiencing")
    print("- Include duration of symptoms")
    print("- Mention any relevant medical history")
    print("--------------------------------")
    
    while True:
        print("\nEnter symptoms (or 'quit' to exit):")
        symptoms = input().strip()
        
        if symptoms.lower() == 'quit':
            print("\nThank you for using the Disease Prediction System!")
            break
            
        if symptoms:
            result = predictor.predict_disease(symptoms)
            
            if isinstance(result, dict):
                print("\n=== Prediction Results ===")
                print(f"Predicted Disease: {result['disease']}")
                print("\nRecognized Symptoms:")
                for symptom in result['recognized_symptoms']:
                    print(f"- {symptom}")
                print("\nNOTE: This is not a medical diagnosis. Please consult a healthcare professional.")
            else:
                print(result)
        else:
            print("\nPlease enter some symptoms.")

if __name__ == "__main__":
    main()

C:\Users\shiva\AppData\Local\Temp\ipykernel_26880\1414441610.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(model_path, map_locat

Initializing Disease Prediction System...
Using device: cuda

Disease Prediction System Ready!
--------------------------------
Tips for describing symptoms:
- Be specific about what you're experiencing
- Include duration of symptoms
- Mention any relevant medical history
--------------------------------

Enter symptoms (or 'quit' to exit):



=== Prediction Results ===
Predicted Disease: Anemia
Confidence: 75.53%

Recognized Symptoms:
- headache

NOTE: This is not a medical diagnosis. Please consult a healthcare professional.

Enter symptoms (or 'quit' to exit):

Thank you for using the Disease Prediction System!
